In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/Users/chris/gitspace/iwrite/tec.asiachrispy.com/data/MNIST_data/', one_hot=True)

import tensorflow as tf
sess = tf.InteractiveSession()

# The input images x will consist of a 2d tensor of floating point numbers

# 784 is the dimensionality of a single flattened 28 by 28 pixel MNIST image, 
# None indicates that the first dimension, corresponding to the batch size, can be of any size
x = tf.placeholder(tf.float32, shape=[None, 784]) 

# each row is a one-hot 10-dimensional vector indicating which digit class (zero through nine) 
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# have 784 input features and 10 outputs
W = tf.Variable(tf.zeros([784,10]))

# b is a 10-dimensional vector,have 10 classes
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

## Predicted Class and Loss Function
y = tf.matmul(x,W) + b

"""
---
"""
## Train the Model
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# for i in range(1000):
#     # load 100 training examples in each training iteration
#     batch = mnist.train.next_batch(100) 
#     # when run, will apply the gradient descent updates to the parameters
#     train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
    
# correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# #   cast to floating point numbers
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
"""
---
"""
## Weight InitializationWeight Initialization
#  One should generally initialize weights with a small amount of noise for symmetry breaking, 
#  and to prevent 0 gradients
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Since we're using ReLU neurons, it is also good practice to initialize them 
# with a slightly positive initial bias to avoid "dead neurons"
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling
# TensorFlow also gives us a lot of flexibility in convolution and pooling operations
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5:卷积，1:输入的通道数目,32:输出的通道数目
b_conv1 = bias_variable([32])

# 最后一维代表图片的颜色通道数(因 为是灰度图所以这里的通道数为1,如果是rgb彩色图,则为3
x_image = tf.reshape(x, [-1,28,28,1]) #把 x 变成一个4d向量,28*28,

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Densely Connected Layer
# 图片尺寸减小到7x7,加入一个有1024个神经元的全连接层,用于处理整个图片
W_fc1 = weight_variable([7 * 7 * 64, 1024]) 
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) # -1就是缺省值，线计算其它为维度，最后总数除以其它几个的乘积，就是这个值。
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## dropout
# 为了减少过拟合,我们在输出层之前加入dropout
# 在训练过程中启用dropout,在测试过程中关闭dropout
keep_prob = tf.placeholder(tf.float32) #一个神经元的输出在dropout 中保持不变的概率
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)#除了可以屏蔽神经元的输出外,还会自动处理神经元输出值的scale

## readout  layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## train annd evaluatee
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) # 用更加 复杂的ADAM优化器来做梯度最速下降

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))